In [2]:
import pandas as pd
from sqlalchemy import inspect, create_engine
import config
host = config.host()
port = config.port()
database = config.database()
user = config.user()
password = config.password()
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")
conn = engine.connect()
inspector = inspect(engine)


In [3]:

jsonList= []
df_dict = {}
new_df = pd.DataFrame()
for table_name in inspector.get_table_names():
    data = pd.read_sql(f"SELECT * FROM \"{table_name}\"", conn)
    fn = f'resources/json_datasets/{table_name}.json'
    data.to_json(fn)
    jsonList.append(fn)
    # print(data)
    df_dict[table_name] = data
    # new_df = pd.concat([new_df,data])


conn.close()

jsonList

['resources/json_datasets/LCA_Disclosure_Data.json',
 'resources/json_datasets/GSEARCH.json',
 'resources/json_datasets/H_1B_Disclosure_Data.json']

In [4]:
df_dict.keys()

dict_keys(['LCA_Disclosure_Data', 'GSEARCH', 'H_1B_Disclosure_Data'])

In [5]:
## concatenate lca and h1b
new_df = pd.DataFrame()
for f in ['LCA_Disclosure_Data', 'H_1B_Disclosure_Data']:
    new_df = pd.concat([new_df,df_dict[f]])
new_df = new_df.mask(new_df.eq('None')).dropna()
new_df.reset_index(inplace=True,drop=True)
new_df.head()

,CASE_NUMBER,RECEIVED_DATE,DECISION_DATE,SOC_TITLE,FULL_TIME_POSITION,BEGIN_DATE,END_DATE,EMPLOYER_NAME,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,WAGE_RATE_OF_PAY_FROM,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY
0,I-200-19268-495825,2019-09-25,2019-10-01,OPERATIONS RESEARCH ANALYSTS,Y,2019-10-07,2022-10-06,"BIZINTEX, INC.",WOODSTOCK,GA,30188,73000.0,Year,72280.0,Year
1,I-200-19268-874666,2019-09-25,2019-10-01,"COMPUTER OCCUPATIONS, ALL OTHER",Y,2019-10-10,2022-10-09,"LOGIC PLANET, INC.",PRINCETON,NJ,8540,78915.0,Year,76419.0,Year
2,I-200-19268-206230,2019-09-25,2019-10-01,COMPUTER SYSTEMS ANALYSTS,Y,2019-10-02,2022-09-01,POLARIS CONSULTING & SERVICES LTD,PISCATAWAY,NJ,08854-6144,87000.0,Year,81890.0,Year
3,I-200-19268-379595,2019-09-25,2019-10-01,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,2019-09-25,2022-09-24,"XENONINFOTEK, INC.",EDISON,NJ,8817,81931.0,Year,81931.0,Year
4,I-200-19268-717286,2019-09-25,2019-10-01,MANAGEMENT ANALYSTS,Y,2019-10-07,2022-10-06,"BIZINTEX, INC.",WOODSTOCK,GA,30188,99000.0,Year,98010.0,Year


In [23]:
totalPW = []
totalWF = []
for j,i in new_df.iterrows():
    if i['WAGE_UNIT_OF_PAY'] != i['PW_UNIT_OF_PAY']:

        ### check for incorrect wage pay using difference
        if not abs((i['WAGE_RATE_OF_PAY_FROM']-i['PREVAILING_WAGE'])/(i['WAGE_RATE_OF_PAY_FROM']+i['PREVAILING_WAGE'])) > 0.5:
            
            avgWage = (i['WAGE_RATE_OF_PAY_FROM']+i['PREVAILING_WAGE'])/2
            unitWage = [i['WAGE_UNIT_OF_PAY'],i['PW_UNIT_OF_PAY']]
            ### Change to longest pay period (this is based on observed pattern of error)
            if 'Year' in unitWage:
                unit = 'Year'
            elif 'Month' in unitWage:
                unit = 'Month'
            elif 'Bi-Weekly' in unitWage:
                unit = 'Bi-Weekly'
            elif 'Week' in unitWage:
                unit = 'Week'
            elif 'Hour' in unitWage:
                unit = 'Hour'
            else:
                print(unitWage)
            i['WAGE_UNIT_OF_PAY'] = unit
            i['PW_UNIT_OF_PAY'] = unit

    if i['WAGE_UNIT_OF_PAY'] == 'Year':
        totalWF.append(i['WAGE_RATE_OF_PAY_FROM'])
    elif i['WAGE_UNIT_OF_PAY'] == 'Month':
        totalWF.append(i['WAGE_RATE_OF_PAY_FROM'] * 12)
    elif i['WAGE_UNIT_OF_PAY'] == 'Week':
        totalWF.append(i['WAGE_RATE_OF_PAY_FROM'] * 52)
    elif i['WAGE_UNIT_OF_PAY'] == 'Hour':
        totalWF.append(i['WAGE_RATE_OF_PAY_FROM'] * 40 * 52)
    elif i['WAGE_UNIT_OF_PAY'] == 'Bi-Weekly':
        totalWF.append(i['WAGE_RATE_OF_PAY_FROM'] * 26)
        
    if i['PW_UNIT_OF_PAY'] == 'Year':
        totalPW.append(i['PREVAILING_WAGE'])
    elif i['PW_UNIT_OF_PAY'] == 'Month':
        totalPW.append(i['PREVAILING_WAGE'] * 12)
    elif i['PW_UNIT_OF_PAY'] == 'Week':
        totalPW.append(i['PREVAILING_WAGE'] * 52)
    elif i['PW_UNIT_OF_PAY'] == 'Hour':
        totalPW.append(i['PREVAILING_WAGE'] * 40 * 52)
    elif i['PW_UNIT_OF_PAY'] == 'Bi-Weekly':
        totalPW.append(i['PREVAILING_WAGE'] * 26)
new_df['WAGE_RATE_OF_PAY_CALCULATED'] = totalWF
new_df['PREVAILING_WAGE_CALCULATED'] = totalPW
fixedTitles = [title.upper() for title in new_df['SOC_TITLE']]
new_df['SOC_TITLE'] = fixedTitles

locs = pd.read_csv('resources\datasets\stateLoc.csv').set_index('state')

wage_mean_df = new_df.groupby(by='EMPLOYER_STATE').median()
listingCnt = new_df.groupby(by='EMPLOYER_STATE').count()
wageDf = pd.DataFrame(wage_mean_df[['WAGE_RATE_OF_PAY_CALCULATED', 'PREVAILING_WAGE_CALCULATED']])
wageDf['LISTING_CNT'] = listingCnt['CASE_NUMBER']
wageDf = locs.join(wageDf,how='outer').dropna()
wageDf.reset_index(inplace=True)
wageDf.rename(columns={'index':'STATE_ABBR'},inplace=True)

constrainDf = new_df[['SOC_TITLE','EMPLOYER_NAME','EMPLOYER_CITY','EMPLOYER_STATE','PREVAILING_WAGE_CALCULATED','WAGE_RATE_OF_PAY_CALCULATED']]

gsearch = df_dict['GSEARCH']
skills = []
for i in gsearch['DESCRIPTION_TOKEN']:
    try:
        skills.append(i.split('/ '))
    except AttributeError:
        skills.append([])
gsearch['SKILLS'] = skills

platform = []
for i in gsearch['VIA']:
    try:
        platform.append(' '.join(i.split(' ')[1:]))
    except AttributeError:
        platform.append('')

gsearch['PLATFORM'] = platform
gsearch = gsearch[['TITLE', 'COMPANY_NAME', 'SKILLS',	'PLATFORM']]

# # new_df is cleaned and concatenated h1b and lca
# # wagedf is infomration concerning wage by state with longitude and latitude
# # locs is coordinate of each state
# # constrainDF is usable information in a smaller file
# # gsearch is cleaned google search data

new_df.to_json('resources/json_datasets/LCA_H1b_Combined.json',orient='records')
wageDf.to_json('resources/json_datasets/wageInfo.json',orient='records')
locs.to_json('resources/json_datasets/locations.json',orient='records')
constrainDf.to_json('resources/json_datasets/jobTitle.json',orient='records')
gsearch.to_json('resources/json_datasets/gsearch.json',orient='records')


In [21]:
wageDf.to_json('resources/json_datasets/wageInfo.json',orient='records')

In [22]:
wageDf

,STATE_ABBR,latitude,longitude,name,WAGE_RATE_OF_PAY_CALCULATED,PREVAILING_WAGE_CALCULATED,LISTING_CNT
0,AK,63.588753,-154.493062,Alaska,93000.00,73725.5,32
1,AL,32.318231,-86.902298,Alabama,80000.00,71947.0,659
2,AR,35.201050,-91.831833,Arkansas,81248.44,69659.0,1395
3,AZ,34.048928,-111.093731,Arizona,82000.00,75982.0,1971
4,CA,36.778261,-119.417932,California,95000.00,87173.0,59873
5,CO,39.550051,-105.782067,Colorado,86576.54,80038.0,2104
6,CT,41.603221,-73.087749,Connecticut,91790.00,82909.0,3321
7,DC,38.905985,-77.033418,District of Columbia,79019.00,72779.2,1093
8,DE,38.910832,-75.527670,Delaware,76200.00,74797.0,2693
9,FL,27.664827,-81.515754,Florida,80891.00,75324.0,8665
